# Transformando os dados

No passo anterior nós criamos as Delta Tables e as salvamos no HDFS.
Entretanto esses dados não estão no formato ideal para realizar as análises.
Os tipos dos dados também não estão apropriados pois todas as colunas foram mapeadas como *string*.
Nesse notebook iremos transformar os dados da coluna *value* em formato JSON para formato tabular utilizando a função *from_json* do Spark.

## Preparando o ambiente

O código abaixo adiciona a **raiz** do projeto, que contém códigos e dados necessários para o "Hands on".

In [1]:
root = '/home/bigdata/jupyterhub'

import sys
sys.path.append(root)

wd = '/delta'

O trecho de código abaixo prepara o ambiente, carregando códigos auxiliares e dados de configuração.O código disponível no pacote *commom.utils* na classe *DataframeUtils* contém vários métodos que facilitam a leitura e escrita dos dados do Postgres. A classe *DataframeUtils* também inicia uma instância do Apache Spark com o Delta Lake integrado ao Spark.

Já o arquivo *config.yaml* tem os dados de acesso ao Postgres e Kafka.

In [2]:
import yaml

from common.utils import DataframeUtils
import pyspark.sql.functions as F

config = yaml.safe_load(open('../config.yaml'))
dfu = DataframeUtils(config)

O trecho abaixo cria o Dataframe no formato **delta** de clientes apontando para o diretório do HDFS.

In [3]:
clientes = dfu. \
  spark(). \
  read. \
  format('delta'). \
  load(f'{wd}/data/clientes-bronze')

Antes de iniciar a extração dos dados, vamos visualizar como eles estão armazenados na *bronze table*.

```
[ ]: clientes.select('value').limit(1).show(truncate=False)
+---------------------------------------------------------------------------------------------------------+
|value                                                                                                    |
+---------------------------------------------------------------------------------------------------------+
|{"city":"SANTA TEREZA DE","client_id":"306162244","cnae_id":"47.29-6-02","defaulting":false,"state":"GO"}|
+---------------------------------------------------------------------------------------------------------+
```

In [4]:
clientes.select('value').limit(1).show(truncate=False)

+---------------------------------------------------------------------------------------------------------+
|value                                                                                                    |
+---------------------------------------------------------------------------------------------------------+
|{"city":"SANTA TEREZA DE","client_id":"306162244","cnae_id":"47.29-6-02","defaulting":false,"state":"GO"}|
+---------------------------------------------------------------------------------------------------------+



O trecho abaixo cria uma view temporária para executar códigos com o Spark SQL

In [5]:
clientes.createOrReplaceTempView('clientes_bronze')

Agora que já sabemos como os dados estão armanenados, podemos realizar a extração das propriedades JSON contidas no campo *value* utilizando a função from_json do Spark. Nesse ponto já vamos alterar o tipo da coluna *defaulting* para boolean.

A extração dos dados segue o mesmo padrão para todas as colunas:
```
from_json(value, '<nome do campo> <tipo do dado>')
```

A função *from_json* retorna um *Row*, por isso precisamos extrair o campo que queremos, o código fica assim:
```
from_json(value, '<nome do campo> <tipo do dado>')[<nome do campo>] as <nome do campo>
```

Na consulta abaixo aplicamos esse padrão de extração em todas as colunas de clientes.

In [6]:
df = dfu.spark().sql("""
select 
  key
, from_json(value, 'client_id string')['client_id'] as client_id
, from_json(value, 'city string')['city'] as city
, from_json(value, 'state string')['state'] as state
, from_json(value, 'cnae_id string')['cnae_id'] as cnae_id
, from_json(value, 'defaulting string')['defaulting'] as defaulting
, max(timestamp) as timestamp
from clientes_bronze
group by 1,2,3,4,5,6
""")

df.printSchema()
df.limit(5)

root
 |-- key: string (nullable = true)
 |-- client_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- cnae_id: string (nullable = true)
 |-- defaulting: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



key,client_id,city,state,cnae_id,defaulting,timestamp
232,58811879,JOAO PESSOA,PB,47.71-7-01,false,2020-06-29 18:16:...
381,58813621,SAO JOSE DO PIA,PI,4771701,false,2020-06-29 18:16:...
562,58813754,TERESINA,PI,4771701,false,2020-06-29 18:16:...
624,58818233,SANTAREM,PA,47.71-7-01,false,2020-06-29 18:16:...
812,21433734,AURORA,CE,4771704,false,2020-06-29 18:16:...


O trecho abaixo escreve no HDFS em formato delta o resultado da consulta acima

In [7]:
df. \
  write. \
  mode('overwrite'). \
  format('delta'). \
  save(f'{wd}/data/clientes-silver')

Agora vamos realizar os mesmos passos para os dados de pedidos. A primeira etapa é criar o Dataframe apontando para o diretório no HDFS.

In [8]:
pedidos = dfu. \
  spark(). \
  read. \
  format('delta'). \
  load(f'{wd}/data/pedidos-bronze')

Agora podemos visualizar como os dados estão armazenados na *bronze table*.

In [9]:
pedidos.select('value').limit(1).show(truncate=False)

+-------------------------------------------------------------------------------------------------------------+
|value                                                                                                        |
+-------------------------------------------------------------------------------------------------------------+
|{"client_id":"20513391","order_date":"2019-08-05T20:10:00.000Z","order_id":"2902005913","salesman_id":"2902"}|
+-------------------------------------------------------------------------------------------------------------+



O trecho abaixo cria a *temp view* para executar consultas usando o Spark SQL.

In [10]:
pedidos.createOrReplaceTempView('pedidos_bronze')

E por fim, a consulta de extração/transformação dos dados de pedidos.
Veja que estamos transformando a coluna **order_date** para o tipo *date* e **order_amount** para o tipo *float*.

In [11]:
df = dfu.spark().sql("""
select 
  key
, from_json(value, 'client_id string')['client_id'] as client_id
, from_json(value, 'order_id string')['order_id'] as order_id
, from_json(value, 'order_date date')['order_date'] as order_date
, from_json(value, 'order_amount float')['order_amount'] as order_amount
, from_json(value, 'salesman_id string')['salesman_id'] as salesman_id
from pedidos_bronze
""")

df.printSchema()
df.limit(5)

root
 |-- key: string (nullable = true)
 |-- client_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_amount: float (nullable = true)
 |-- salesman_id: string (nullable = true)



key,client_id,order_id,order_date,order_amount,salesman_id
0,20513391,2902005913,2019-08-05,null,2902
1,20513391,2902005912,2019-08-05,1613.644,2902
2,20513391,2503005207,2019-10-03,745.937,2503
3,20513391,2503004969,2019-07-04,1507.87,2503
4,20513391,2503005208,2019-10-03,null,2503


O trecho abaixo escreve o resultado da consulta de pedidos no HDFS em formato delta.

In [12]:
df. \
  write. \
  mode('overwrite'). \
  format('delta'). \
  save(f'{wd}/data/pedidos-silver')

# Exercício

Agora vamos colocar em prática o que aprendemos sobre a extração de dados.
Crie um código para extrair os dados de itens de pedidos e realizar as devidas alterações dos tipos de dados.
De forma geral, o código deverá realizar as seguintes etapas:

1. Criar o Dataframe apontando para o HDFS (/delta/data/itens-bronze)
2. Visualizar como os dados estão armazenados na tabela bronze.
3. Realizar a extração usando o Spark SQL.
4. Escrever os novos dados em /delta/data/itens-silver.

In [3]:
# Comece criando o dataframe
itens = dfu. \
  spark(). \
  read. \
  format('delta'). \
  load(f'{wd}/data/itens-bronze')

In [4]:
# Veja como os dados estão armazenados
itens.select('value').limit(1).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                           |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"client_id":"20513391","items_count":1,"list_price":5.0205,"order_date":"2019-07-04T04:00:00.000Z","order_id":"2503004970","product_id":"11925","sale_price":5.0205,"salesman_id":"2503","supplier_id":"11967"}|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
# Crie uma temp view
itens.createOrReplaceTempView('itens_bronze')

In [6]:
# Realize a extração usando o Spark SQL
dfi = dfu.spark().sql("""
select 
  key
, from_json(value, 'client_id string')['client_id'] as client_id
, from_json(value, 'order_id string')['order_id'] as order_id
, from_json(value, 'order_date date')['order_date'] as order_date
, from_json(value, 'items_count integer')['items_count'] as items_count
, from_json(value, 'list_price float')['list_price'] as list_price
, from_json(value, 'sale_price float')['sale_price'] as sale_price
, from_json(value, 'salesman_id string')['salesman_id'] as salesman_id
, from_json(value, 'product_id string')['product_id'] as product_id
, from_json(value, 'supplier_id string')['supplier_id'] as supplier_id
from itens_bronze
""")

dfi.printSchema()
dfi.limit(5)

root
 |-- key: string (nullable = true)
 |-- client_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- items_count: integer (nullable = true)
 |-- list_price: float (nullable = true)
 |-- sale_price: float (nullable = true)
 |-- salesman_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- supplier_id: string (nullable = true)



key,client_id,order_id,order_date,items_count,list_price,sale_price,salesman_id,product_id,supplier_id
0,20513391,2503004970,2019-07-04,1,5.0205,5.0205,2503,11925,11967
1,20513391,2503004969,2019-07-04,40,1.4574,1.3117,2503,18075,6000
2,20513391,2503004969,2019-07-04,30,1.8989,1.5382,2503,18083,6000
3,20513391,2503004969,2019-07-04,48,0.9069,0.7346,2503,18103,6000
4,20513391,2902005913,2019-08-05,1,3.2,3.2,2902,210157,11265


In [7]:
# Escreva o resultado em /delta/data/itens-silver
dfi. \
  write. \
  mode('overwrite'). \
  format('delta'). \
  save(f'{wd}/data/itens-silver')